
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: gold ; color : 'black'; text-align: left; border-radius: 2px 2px;pad:4px;"> Problem Statement </h2>

<strong>Problem Statement:</strong>

We have a lot of customer on our InvestorAI platform. Whenever they login in our app and view anything, we get pings from their mobile phone indicating that they are using the app.
<br>
You have been provided with 3 weeks of training data and 1 week of test data.
<br>
<br>
<strong>About Data and Variables</strong>

Training data contains id, gender, age, number of kids the customer has and all the pings that have been received (during the training data period).
<br><br>
<strong>Objective:</strong>

We want to predict how many hours the customer will be online / using our app on a given day. So the test data contains customer id, and date (during the test data period).
The test data also contains the actual online hours, which is what your model should predict.

<strong>Evaluation Metric:</strong>

We will be looking at Root Mean Squared Error or RMSE for short (lower the better) to see how good your model is.

<strong> Special Note:</strong>

We have a separate held out test dataset against which we will evaluate your model for overfitting, underfitting, etc.

<strong>Initial Insights: </strong>

1. Looks like we are handling a raw data as custumer database and the activity are seperated out in the data itself. So few assumpitions and data cleaning could be needed ?
2. Clearly given in the problem statement that we are going to handle a time related data for each customer.  This could be a multivariante time series problem with multiple timeseries. So modeling stategy and model building need to  be done to capture patterns, ( lets assume)
3. Error metric is root mean squred error so its sensitive to small changes, can I optimize my model for better metric, root mean squared log error which is more reponsive to very fine changes?
4. As there is another secret hold out set for validation for overfitting and underfitting, so can I hold few datapoints to check this prior?

<p style = "font-size:25px"><strong>Lets Dive Into The Data....</strong></p>


<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: gold ; color : 'black'; text-align: left; border-radius: 2px 2px;pad:4px;"> Data and Initial Stats </h2>

loading various module I am planning to use for this assignment

In [25]:
#importing modules

import warnings 
warnings.filterwarnings('ignore')

import time
t = time.time()

print('Importing startred...')

# base libraries
import os
import numpy as np
import pandas as pd
import re
from scipy import stats
from random import randint
from datetime import datetime


# visualization libraries
import matplotlib.pyplot as plt
import matplotlib 
%matplotlib inline
import seaborn as sns
import missingno as msno


# preprocessing libraries
from sklearn.model_selection import (TimeSeriesSplit,
                                     GridSearchCV,
                                     RandomizedSearchCV)

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler, 
                                   MinMaxScaler, 
                                   OrdinalEncoder)

from sklearn.feature_selection import SelectFromModel


# metrics
from sklearn.metrics import (mean_squared_error, 
                             r2_score, 
                             mean_absolute_error)
from sklearn.metrics import make_scorer


# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso, 
                                  ridge_regression,
                                  LinearRegression)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (AdaBoostRegressor, 
                              RandomForestRegressor,
                              VotingRegressor, 
                              GradientBoostingRegressor)
from xgboost import XGBRegressor
from lightgbm import (LGBMRegressor,
                      early_stopping)

from sklearn.base import clone ## sklearn base models for stacked ensemble model


#Interpretiability of the model
import shap
import eli5
from eli5.sklearn import PermutationImportance


## misea
from sklearn.pipeline import make_pipeline



print('Done,All the required modules are imported. Time elapsed: {}sec'.format(time.time()-t))

Importing startred...
Done,All the required modules are imported. Time elapsed: 0.0060040950775146484sec


<p style = "font-size:25px"><strong>Lets load the data and visualize it for basic understanding....</strong></p>

In [22]:
# loading data

customer_df = pd.read_csv('./data/customers.csv', delimiter = ',', encoding = 'utf-8')
pings_df = pd.read_csv('./data/pings.csv', delimiter = ',', encoding = 'utf-8')
test_df = pd.read_csv('./data/test.csv', delimiter = ',', encoding = 'utf-8')

In [23]:
customer_df.head(2)

,id,gender,age,number_of_kids
0,979863,MALE,26,2
1,780123,MALE,60,2


In [24]:
pings_df.head(2)

,id,timestamp
0,899313,1496278800
1,373017,1496278800


In [26]:
test_df.head(2)

,id,date,online_hours
0,979863,28/06/17,7
1,979863,27/06/17,9
